In [ ]:
'''
Derek's Notes:

I chopped the heck out of the file provided to us by James Clark
in hopes that we could get it to work for our own code. It's a total
hack job, and I'm doing a disservice to Mr. Clark, but all I care about
right now is getting it to work in an attempt to learn more about
elegantly transitioning data from NINJA format into what we'll need
for LALsim. Again, this is a hack job, but hopefully it'll help inform
how to make the next attempt better.
'''

#!/usr/bin/env python
# -*- coding:utf-8 -*-
# Copyright (C) 2015-2016 James Clark <clark@physics.umass.edu>
#
# This program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 2 of the License, or
# (at your option) any later version.
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License along
# with this program; if not, write to the Free Software Foundation, Inc.,
# 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.

from __future__ import division
import os,sys
import hashlib
import numpy as np
import glob

import pycbc.types
from pycbc.waveform import utils as wfutils
from pycbc import pnutils
#import pycbc.filter
import scipy.signal
import lal


import h5py
import romspline as romSpline

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Waveform Generation
#

datahome='/home/dwhite/GWPAC'
waveform_list=list()

wave_dict=dict()
wave_dict['datadir']=datahome+'/Giacomazzo/Qeven_Detector_Radius_400.00_l2_m2.txt'
wave_dict['mass1']=1.22
wave_dict['mass2']=1.22
wave_dict['spinz']=0.0
waveform_list.append(wave_dict)

In [ ]:
for wave_dict in waveform_list:

    datadir=wave_dict['datadir']
    mass1=wave_dict['mass1']
    mass2=wave_dict['mass2']
    spinz=wave_dict['spinz']
    total_mass=mass1+mass2
    
    print datadir
    
    globpattern=glob.glob(datadir)[0].split('/')[-1]
    print globpattern
    
    simname=globpattern.replace('.txt','')
    print simname
    
    h5file=simname+'_final.h5'
    print h5file    

    ninjafiles=glob.glob(datadir)
    print ninjafiles
    
    print''

    # Hardcoded, fixed delta_t is fine for Bauswein et al:
    delta_t = 1./16384
    f_lower_hz = 700.0 

    #startFreqHz = startFreq / (lal.TWOPI * massTotal * lal.MTSUN_SI)
    f_lower = f_lower_hz * (lal.TWOPI * total_mass * lal.MTSUN_SI)

    with h5py.File(h5file,'w') as fd:

        #
        # Set metadata
        #

        mchirp, eta = pnutils.mass1_mass2_to_mchirp_eta(mass1, mass2)
        fd.attrs.create('NR_group', 'GWPAC_NS')
        fd.attrs.create('name', 'GWPAC:BNS:%s'%simname)
        hashtag = hashlib.md5()
        hashtag.update(fd.attrs['name'])
        fd.attrs.create('hashtag', hashtag.digest())
        fd.attrs.create('f_lower_at_1MSUN', f_lower)
        fd.attrs.create('eta', eta)
        fd.attrs.create('spin1x', 0.0)
        fd.attrs.create('spin1y', 0.0)
        fd.attrs.create('spin1z', spinz)
        fd.attrs.create('spin2x', 0.0)
        fd.attrs.create('spin2y', 0.0)
        fd.attrs.create('spin2z', spinz)


        # XXX HARDCODING for non-spinning / aligned-spin
        fd.attrs.create('LNhatx', 0.0)
        fd.attrs.create('LNhaty', 0.0)
        fd.attrs.create('LNhatz', 1.0)

        fd.attrs.create('nhatx', 1.0)
        fd.attrs.create('nhaty', 0.0)
        fd.attrs.create('nhatz', 0.0)


        fd.attrs.create('coa_phase', 0.0)
        fd.attrs.create('mass1', mass1/total_mass)
        fd.attrs.create('mass2', mass2/total_mass)


        for ninjafile in ninjafiles:
            
            # set l and m values
            l=2.0
            
            if ninjafile == '/home/dwhite/GWPAC/mp_psi4_l2_m-2_r400.00.asc':
                m=-2.0
            else:
                m=2.0

            timesGeom, hplusGeom, hcrossGeom = np.loadtxt(ninjafile, unpack=True)

            #
            # Reverse engineer for sanity check and resampling
            #
            scalefac = lal.MRSUN_SI / (10e6*lal.PC_SI)
            times = (timesGeom * lal.MTSUN_SI)

            native_delta_t = np.diff(times)[0]
            hplus = hplusGeom*scalefac*total_mass*np.sqrt(2)
            hcross = hcrossGeom*scalefac*total_mass*np.sqrt(2)

            massMpc = total_mass*scalefac

            hplusMpc  = pycbc.types.TimeSeries(hplus/massMpc, delta_t=delta_t)
            hcrossMpc = pycbc.types.TimeSeries(hcross/massMpc, delta_t=delta_t)
            times_M = (times / (lal.MTSUN_SI * total_mass))
     
            HlmAmp   = wfutils.amplitude_from_polarizations(hplusMpc,
                    hcrossMpc).data
            HlmPhase = wfutils.phase_from_polarizations(hplusMpc, hcrossMpc).data 

    #       if l!=2 or abs(m)!=2:
    #           HlmAmp = np.zeros(len(HlmAmp))
    #           HlmPhase = np.zeros(len(HlmPhase))
     
            print 'fitting spline...'
            sAmph = romSpline.ReducedOrderSpline(times_M, HlmAmp, verbose=False)
            sPhaseh = romSpline.ReducedOrderSpline(times_M, HlmPhase, verbose=False)
          
            gramp = fd.create_group('amp_l%d_m%d' %(l,m))
            sAmph.write(gramp)
            
            grphase = fd.create_group('phase_l%d_m%d' %(l,m))
            sPhaseh.write(grphase)
            
            print 'spline created'